This notebook demonstrates how to run a T2 analysis unit (in this case sncosmo) on a list (tar) of transient views. Transient views are summaries of all information regarding a transient exported from the core Ampel DB. Using these thus do not require an active local database. 

In [ ]:
import re, os
import ampel.utils.json_serialization as ampel_serialization
import logging
import gzip
import importlib
from ampel.ztf.pipeline.common.ZTFUtils import ZTFUtils
import ampel.contrib.hu.t2.T2SNCosmo as T2SNCosmo
import sncosmo

In [ ]:
# Paths etc (edit as needed)
# Transient view collection
tvdir = '/home/jnordin/data/transientViews/'     # *** Change path *** # 
fname = tvdir+'tnscomplete_aug28.json.gz'        # *** Change path *** #
outdir = '/home/jnordin/tmp/sncosmo/'
# Logging    # 
logpath = '/home/jnordin/tmp/' # *** Change path *** # 
# Which model to fit
fitmodel = 'salt2'
#fitmodel = "kilo_mej0.02_lant15_theta90"
# Limit fit to subset of sne
fit_sne = []

In [ ]:
# Get a TransientView iterator
if re.search('gz$',fname):
    tv_iterator = ampel_serialization.load(gzip.open(fname,'rb'))
else:    
    tv_iterator = ampel_serialization.load(open(fname))
tvlist = []
try:
    for tv in tv_iterator:
        tvlist.append(tv)
except EOFError as e:
    print('... prob just the usual missing eof marker. Check that the number agrees')
print('Found %s transient views'%(len(tvlist)))

In [ ]:
# Create a logger
logger = logging.getLogger()
logger.setLevel((logging.INFO))
handler = logging.FileHandler(os.path.join(logpath, 'run_t2sncosmo.log'))
logger.addHandler(handler)

 What lightcurve fit is being carried out is detrmined by the run (and base) configurations which are supplied to the analysis unit.

In [ ]:
# The base config are channel parameters provided to the analysis unit at init. Usually not needed.
base_config = {}

In [ ]:
# The run config are parameters used by the analysis unit for each fit. For sncosmo these are usually model dependent
if fitmodel=='salt2':
    run_config = {
            "model": "salt2",
            "with_upper_limits": False,
            "sncosmo_kwarg": {},
            "min_det" : 8,
            "jd_reject_sigma" : 5,
            "min_duration" : 10,
            "max_duration" : 90
            }
if fitmodel=="kilo_mej0.02_lant15_theta90":
    run_config = {
            "model": "kilo_mej0.02_lant15_theta90",
            "with_upper_limits": False,
            "sncosmo_kwarg": {"bounds":{'z':[0,0.05]}},
            "min_det" : 8,
            "jd_reject_sigma" : 5,
            "min_duration" : 3,
            "max_duration" : 16
        }

In [ ]:
# During development
importlib.reload(sncosmo)
importlib.reload(T2SNCosmo)

In [ ]:
# This is where an instance of the T2 analysis module is created
myt2 = T2SNCosmo.T2SNCosmo(logger, base_config)

In [ ]:
%%time
for tv in tvlist:
    ztf_name = ZTFUtils.to_ztf_id(tv.tran_id)    
    if len(fit_sne)>0 and not ztf_name in fit_sne : continue
    print('Trying to fit model %s to %s'%(fitmodel,ztf_name))    
    run_config['savefile'] = outdir+'%s.pdf'%(ztf_name)   
    lc = tv.get_latest_lightcurve()
    out = myt2.run(lc, run_config)
    if len(out)==0:
        print('   no fit done (insufficient data?)')
    else:
        print('   model fit output: ',out['fit_results'])
